In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")


## API Documentation

- https://developers.arcgis.com/rest/services-reference/enterprise/feature-service/
- https://data-nifc.opendata.arcgis.com/datasets/nifc::wildland-fire-incident-locations/about


In [ ]:
api_spec = """
openapi: 3.1.1
info:
  title: Wildland Fire Incident Locations
  summary: Point Locations for all wildland fires in the United States reported to the IRWIN system.
  version: 1.0.0
servers:
  - url: https://services3.arcgis.com/T4QMspbfLg3qTGWY/ArcGIS/rest/services/WFIGS_Incident_Locations
paths:
  /FeatureServer/0/query:
    get:
      description: >
        The query operation queries a feature service resource and returns either a feature set for each layer in the query, a count of features for each layer (if returnCountOnly is set to true ), or an array of feature IDs for each layer in the query (if returnIdsOnly is set to true ).

        Some of the fields in this layer are:
        - IrwinID: Unique identifier assigned to each incident record in IRWIN.
        - FinalAcres: Reported final acreage of incident.
        - FireDiscoveryDateTime: The date and time a fire was reported as discovered or confirmed to exist.  May also be the start date for reporting purposes.
        - IncidentTypeCategory: The Event Category is a sub-group of the Event Kind code and description. The Event Category breaks down the Event Kind into more specific event categories. Values are Wildfire (WF), Prescribed Fire (RX), or Incident Complex (CX).
        - InitialLatitude: The latitude of the initial reported point of origin specified in decimal degrees.
        - InitialLongitude: The longitude of the initial reported point of origin specified in decimal degrees.
        - POOState: The State alpha code identifying the state or equivalent entity at point of origin.

      parameters:
        - name: where
          in: query
          description: >
            An SQL-92 WHERE clause used to filter the features.

            Supported SQL-92 operators include: `<=`, `>=`, `<`, `>`, `=`, `!=`, `<>`, `LIKE`, 
            `AND`, `OR`, `IS`, `IS NOT`, `IN`, `NOT IN`, and `BETWEEN`.

            Usage Notes:
              - The value for this parameter (where) needs to be percent-encoded.
              - Don't quote the entire clause (value) just quote string literals that are part of the expression.
              - Use %27 (single quote) for the string fields.
              - Dont use %22 (double quote).

            Field-Specific Information:
            - POOState: ISO 3166-2 formatted string representing the state where the point of origin is located.

          required: false
          schema:
            type: string
            examples:
              cawildfires:
                value: "POOState = 'US-CA' AND IncidentTypeCategory = 'WF'"
                summary: Wildfires in California
              largefires:
                value: "FinalAcres > 1000"
                summary: Fires larger than 1000 acres
              2023fires:
                value: FireDiscoveryDateTime >= DATE '2023-01-01'
                summary: Fires from 2023 and onward

        - name: outFields
          in: query
          description: Comma-separated list of fields to include in the response.
          required: false
          schema:
            type: string
            example: "POOState,IncidentTypeCategory,FinalAcres,FireDiscoveryDateTime"

        - name: returnGeometry
          in: query
          description: If true, the result includes the geometry associated with each feature returned.
          required: false
          schema:
            type: boolean
            default: true

        - name: returnCountOnly
          in: query
          description: If true, the response only includes the count (number of features/records) that would be returned by a query.
          required: false
          schema:
            type: boolean
            default: false

        - name: resultOffset
          in: query
          description: This option can be used for fetching query results by skipping the specified number of records and starting from the next record (that is, resultOffset + 1).
          required: false
          schema:
            type: number
            default: 0

        - name: resultRecordCount
          in: query
          description: This option can be used for fetching query results up to the resultRecordCount specified.
          required: false
          schema:
            type: number
            default: 2000

        - name: f
          in: query
          description: The response format.
          required: false
          schema:
            type: string
            enum: [html, json, geojson]
            default: html
            example: f=geojson
"""

## APIChain

- https://python.langchain.com/api_reference/langchain/chains/langchain.chains.api.base.APIChain.html

In [ ]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain.chains.api.prompt import API_URL_PROMPT
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.utilities.requests import TextRequestsWrapper
from langchain_core.messages import BaseMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableConfig
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import tools_condition, ToolNode

# NOTE: There are inherent risks in giving models discretion
# to execute real-world actions. We must "opt-in" to these
# risks by setting allow_dangerous_request=True to use these tools.
# This can be dangerous for calling unwanted requests. Please make
# sure your custom OpenAPI spec (yaml) is safe and that permissions
# associated with the tools are narrowly-scoped.
ALLOW_DANGEROUS_REQUESTS = True

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

toolkit = RequestsToolkit(
    requests_wrapper=TextRequestsWrapper(headers={}),  # no auth required
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUESTS,
)
tools = toolkit.get_tools()

api_request_chain = (
    API_URL_PROMPT.partial(api_docs=api_spec)
    | llm.bind_tools(tools, tool_choice="any")
)

class ChainState(TypedDict):
    """LangGraph state."""

    messages: Annotated[Sequence[BaseMessage], add_messages]

async def acall_request_chain(state: ChainState, config: RunnableConfig):
    last_message = state["messages"][-1]
    response = await api_request_chain.ainvoke(
        {"question": last_message.content}, config
    )
    return {"messages": [response]}

graph_builder = StateGraph(ChainState)
graph_builder.add_node("call_tool", acall_request_chain)
graph_builder.add_node("execute_tool", ToolNode(tools))
graph_builder.add_edge(START, "call_tool")
graph_builder.add_edge("call_tool", "execute_tool")
graph_builder.add_edge("execute_tool", END)

chain = graph_builder.compile()

In [ ]:
from IPython.display import Image, display
display(Image(chain.get_graph().draw_mermaid_png()))

## Icons

Icons from [icons8](https://icons8.com/):
- https://icons8.com/icons/set/racoon


In [ ]:
import gradio as gr
from langchain_core.messages import HumanMessage, SystemMessage

async def respond(message, history):
    events = chain.astream(
        {"messages": [HumanMessage(content=message)]},
        stream_mode="values",
    )
    async for event in events:
        msg = event["messages"][-1]

        if isinstance(msg, HumanMessage):
            history.append(gr.ChatMessage(role="user", content=msg.pretty_repr()))

        else:
            history.append(gr.ChatMessage(role="assistant", content=msg.pretty_repr()))

    return "", history

with gr.Blocks(css="button[aria-label='Clear'] { display: none; }") as demo:
    chatbot = gr.Chatbot(
        type="messages",
        layout="bubble",
        show_label=False,
        avatar_images=(None, "icons8-racoon-48.png")
    )
    textbox = gr.Textbox(
        submit_btn=True,
        show_label=False,
        placeholder="Start typing to begin"
    )
    textbox.submit(respond, [textbox, chatbot], [textbox, chatbot])

# Fixes a RuntimError about asynchronous tasks being attached to a different loop.
llm.validate_environment()

demo.launch()

# Get data for 5 wildfires from california discovered since 2023. Format with geojson. Include the time of discovery, and and irwin id in the output and return geometry.


## Satellite Images

- https://skywatch.com/arcgis-pro-add-in-download/
- https://skyfi.com


In [ ]:
import requests

url = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/ArcGIS/rest/services/WFIGS_Incident_Locations/FeatureServer/0/query"

params = {
    "where": "POOState = 'US-CA' AND IncidentTypeCategory = 'WF' AND FireDiscoveryDateTime >= DATE '2023-01-01' AND FinalAcres > 1",
    "outFields": "SourceOID, GlobalID,FireDiscoveryDateTime,InitialLongitude,InitialLatitude",
    "f": "json"
}

response = requests.get(url, params=params)


In [ ]:
import pandas as pd
df = pd.DataFrame(list(map(lambda x: x["attributes"], response.json()["features"])))

df.head()

# df.to_csv("incidents.csv")
df[df["GlobalID"] == "a123d2f9-36ab-4226-9031-6b075697ac90"]

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()


In [ ]:
def get_thumb_url(timestamp, longitude, latitude, cloud_threshold=20):    
    collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    point = ee.Geometry.Point([longitude, latitude])
    incident_date = ee.Date(timestamp)

    image = (collection
             .filterBounds(point)
             .filterDate(incident_date, '2100-01-01')
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_threshold))
             .sort('system:time_start', True)
             .first())

    if not image:
        raise ValueError("No suitable image found for the given parameters.")
    
    buffer = point.buffer(300)  # 300-meter buffer
    clipped_image = image.clip(buffer.bounds())

    params = {
        'min': 0,
        'max': 4000,
        'bands': ['B4', 'B3', 'B2'],
        'dimensions': [350, 350]
    }

    thumbnail_url = clipped_image.getThumbURL(params)
    return thumbnail_url


In [ ]:
failed = []

for index, global_id, timestamp, longitude, latitude in df.itertuples():
    try:
        url = get_thumb_url(timestamp, longitude, latitude)
        response = requests.get(url)
        response.raise_for_status()
        with open(f"images/{global_id}.png", 'wb') as f:
            f.write(response.content)
        print(f"SUCCESS: {global_id}")
    except:
        print(f"FAILURE: {global_id}")
        failed.append(index)
